In [ ]:
import pandas as pd
import json
import os

# --- Configuration ---
INPUT_FILE = 'transactions.txt'
OUTPUT_FILE = 'transactions_processed.csv'
# ---------------------

def prepare_data_for_snowflake(input_path, output_path):
    """Loads JSON data, cleans types, sorts chronologically, and saves to CSV."""
    print(f"Loading data from {input_path}...")
    
    # Load data line by line from the text file
    data = []
    try:
        with open(input_path, 'r') as f:
            for line in f:
                # Handle potential empty lines or malformed JSON gracefully
                if line.strip():
                    data.append(json.loads(line))
    except FileNotFoundError:
        print(f"Error: The file '{input_path}' was not found. Please create it.")
        return
    
    if not data:
        print("Error: The input file is empty or contains no valid JSON.")
        return

    df_raw = pd.DataFrame(data)

    # Renaming and standardizing critical columns
    df_raw.rename(columns={
        'accountNumber': 'ACCOUNT_ID',
        'transactionDateTime': 'TRANSACTION_TS',
        'transactionAmount': 'AMOUNT',
        'merchantName': 'MERCHANTNAME',
        'cardPresent': 'CARD_PRESENT' # Standardize the column name
    }, inplace=True)

    # Type Conversion (Senior DS Focus: Ensure data integrity)
    df_raw['TRANSACTION_TS'] = pd.to_datetime(df_raw['TRANSACTION_TS'])
    df_raw['AMOUNT'] = df_raw['AMOUNT'].astype(float)
    
    # Sort data chronologically - ESSENTIAL for accurate window function calculations
    df_raw.sort_values(by='TRANSACTION_TS', inplace=True)
    
    # Select only the relevant columns for the raw table to keep it clean
    raw_cols = ['ACCOUNT_ID', 'TRANSACTION_TS', 'AMOUNT', 'MERCHANTNAME', 'CARD_PRESENT',
                'merchantCategoryCode', 'posEntryMode', 'posConditionCode']
    
    # Save the cleaned, ordered data to CSV
    df_raw[raw_cols].to_csv(output_path, index=False)
    print(f"Successfully processed {len(df_raw)} records and saved to {output_path}")

In [3]:
prepare_data_for_snowflake(INPUT_FILE, OUTPUT_FILE)

Loading data from transactions.txt...
Successfully processed 786363 records and saved to transactions_processed.csv


In [3]:
pip install snowflake-snowpark-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\nandi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from snowflake.snowpark.session import Session
from sklearn.ensemble import IsolationForest
import joblib

# --- 1. Define Snowflake Connection Parameters ---
# NOTE: Replace these placeholders with your actual Snowflake credentials
SNOWFLAKE_CONN_PARAMS = {
    "account" :"MBYCHCU-LPC98923",
    "user" : "NANZ",
    "password" : "XXXXXXXXXXXXXXXX",
    "warehouse": "COMPUTE_WH",
    "database" :"FINANCIAL_ANALYTICS",
    "schema": "FRAUD_DETECTION"
}



Successfully connected to Snowflake.
Training on 773107 records after dropping 13256 initial rows.
Isolation Forest Model trained successfully.
Model saved locally as 'isolation_forest_model_v1.joblib'


In [ ]:
# --- 2. Define Features for Training ---
MODEL_FEATURES = [
    'AMOUNT', 
    'AMT_1HR_ACCT', 
    'COUNT_24HR_ACCT', 
    'HOUR_OF_DAY',
    'NEW_MERCHANT_COUNT_7D', 
    'AMT_Z_SCORE_30D', 
    'CARD_PRESENT_RATIO_7D'
]

# --- 3. Connect and Pull Data ---
try:
    session = Session.builder.configs(SNOWFLAKE_CONN_PARAMS).create()
    print("Successfully connected to Snowflake.")

    # Pull data from the V_MODEL_FEATURES view
    sql_query = f"SELECT {', '.join(MODEL_FEATURES)} FROM V_MODEL_FEATURES"
    
    # Use Snowpark to get the data into a Pandas DataFrame
    snowpark_df = session.sql(sql_query)
    df_train = snowpark_df.to_pandas()
    
    session.close()

except Exception as e:
    print(f"Error connecting to Snowflake or pulling data: {e}")
    # You would typically have a more robust error handling mechanism
    exit()



In [ ]:
# --- 4. Data Pre-processing (Handling NaNs) ---
# Isolation Forest cannot handle NaN values. NaNs arise from initial transactions
# where the window functions (like Z-score) don't have enough history yet.
X_train = df_train[MODEL_FEATURES].dropna() 
print(f"Training on {len(X_train)} records after dropping {len(df_train) - len(X_train)} initial rows.")

# --- 5. Train the Isolation Forest Model ---
# Concept: The contamination parameter. We set this to the expected fraction of anomalies (0.1%).
# Tuning this value later is key to reducing your 20% false positive rate.
iso_forest = IsolationForest(
    contamination=0.001, # 0.1% expected fraud rate
    random_state=42, 
    n_estimators=100
)

iso_forest.fit(X_train)
print("Isolation Forest Model trained successfully.")

# --- 6. Save the Model ---
MODEL_FILENAME = 'isolation_forest_model_v1.joblib'
joblib.dump(iso_forest, MODEL_FILENAME)
print(f"Model saved locally as '{MODEL_FILENAME}'")